In [1]:
import pandas as pd
import numpy as np
import torch 
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
import glob, os, pickle
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

# Relationship Between Patches in a WSI 
- Need to figure out how to turn a WSI into a graph, and how to know the relationships between different patches
- If the x,y coords represent the center of each patch, I suppose what we can say is that adjacent patches need to be sqrt(2x^2) away if they are truly neighboring patches. 
- Apparently, two nodes might be neighbors if their embeddings are sufficiently similar. 
- Josh says that if patches are sqrt(2)*256 away from each other, then we should consider them neighbors 

In [4]:
df = pd.read_pickle("/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/scc_tumor_data/prelim_patch_info_v2/109_A1c_ASAP_tumor_map.pkl")

In [5]:
df

,ID,x,y,patch_size,annotation,y_true,inflamm,scc
0,109_A1c_ASAP,1024,16640,256,0,0,0,0
1,109_A1c_ASAP,1280,15872,256,0,0,0,0
2,109_A1c_ASAP,1280,16128,256,0,0,0,0
3,109_A1c_ASAP,1280,16384,256,0,0,0,0
4,109_A1c_ASAP,1280,16640,256,0,0,0,0
...,...,...,...,...,...,...,...,...
41290,109_A1c_ASAP,98560,25600,256,0,0,0,0
41291,109_A1c_ASAP,98560,25856,256,0,0,0,0
41292,109_A1c_ASAP,98560,26112,256,0,0,0,0
41293,109_A1c_ASAP,98560,26368,256,0,0,0,0


In [6]:
#create node -> (x,y) map 
pos_map = {}

In [7]:
for index, row in tqdm(df.iterrows()):
    pos_map[index] = (row["x"], row["y"])

41295it [00:01, 30492.68it/s]


# Create edge adj. list
- Here we will use a library because n^2 is too slow
- I am relying on the standard indexing in the df. Meaning, the first row should be patch 1, the second should be patch 2, and so on... 

In [8]:
import dgl
import torch

In [9]:
nodes = torch.tensor([])

In [10]:
for i in tqdm(pos_map):
    x = torch.tensor([[float(pos_map[i][0]), float(pos_map[i][1])]])
    nodes = torch.cat((nodes, x), 0)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41295/41295 [00:00<00:00, 43656.97it/s]


In [11]:
len(nodes)

41295

In [12]:
# create graph where points sqrt(2)*256 away from each other are considered neighbors 
r_g, dist = dgl.radius_graph(nodes, 256*(2**(1/2)), get_distances=True) 

In [13]:
r_g.edges()

(tensor([    0,     0,     0,  ..., 41294, 41294, 41294]),
 tensor([    3,     4,     5,  ..., 41229, 41230, 41293]))

In [14]:
adj_list = list(zip(list(r_g.edges()[0]), list(r_g.edges()[1])))  #this is the adj list 

In [15]:
for i in tqdm(range(len(adj_list))): #converting everything from tensors to ints 
    adj_list[i] = [adj_list[i][0].item(), adj_list[i][1].item()]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 326540/326540 [00:00<00:00, 709379.03it/s]


In [16]:
adj_list

[[0, 3],
 [0, 4],
 [0, 5],
 [1, 2],
 [1, 12],
 [1, 13],
 [1, 14],
 [2, 1],
 [2, 3],
 [2, 13],
 [2, 14],
 [2, 15],
 [3, 0],
 [3, 2],
 [3, 4],
 [3, 14],
 [3, 15],
 [3, 16],
 [4, 0],
 [4, 3],
 [4, 5],
 [4, 15],
 [4, 16],
 [4, 17],
 [5, 0],
 [5, 4],
 [5, 6],
 [5, 16],
 [5, 17],
 [5, 18],
 [6, 5],
 [6, 7],
 [6, 17],
 [6, 18],
 [6, 19],
 [7, 6],
 [7, 8],
 [7, 18],
 [7, 19],
 [7, 20],
 [8, 7],
 [8, 9],
 [8, 19],
 [8, 20],
 [8, 21],
 [9, 8],
 [9, 10],
 [9, 20],
 [9, 21],
 [9, 22],
 [10, 9],
 [10, 21],
 [10, 22],
 [10, 23],
 [11, 12],
 [11, 31],
 [11, 32],
 [11, 33],
 [12, 1],
 [12, 11],
 [12, 13],
 [12, 32],
 [12, 33],
 [12, 34],
 [13, 1],
 [13, 2],
 [13, 12],
 [13, 14],
 [13, 33],
 [13, 34],
 [13, 35],
 [14, 1],
 [14, 2],
 [14, 3],
 [14, 13],
 [14, 15],
 [14, 34],
 [14, 35],
 [14, 36],
 [15, 2],
 [15, 3],
 [15, 4],
 [15, 14],
 [15, 16],
 [15, 35],
 [15, 36],
 [15, 37],
 [16, 3],
 [16, 4],
 [16, 5],
 [16, 15],
 [16, 17],
 [16, 36],
 [16, 37],
 [16, 38],
 [17, 4],
 [17, 5],
 [17, 6],
 [17, 16],

In [17]:
adj_list = torch.tensor(adj_list)

In [18]:
adj_list.shape

torch.Size([326540, 2])

# Create a graph data object 
- Need to use torch.geometric here
- Also need to define the model class
- Also need to get embeddings for each patch here 
- Also need the y matrix
- We need to save all graph data objects in a seperate directory, and get their file location and map them to the meta file
- The idea is that we will save all of this to a df that will contain columns= ["sample_id", "file_loc"] 

## Model Class

In [19]:
#define the model class 
model = torch.hub.load('pytorch/vision', 'resnet50')
model.fc = nn.Sequential(nn.Linear(2048, 100), 
                         nn.ReLU(), 
                         nn.Dropout(p=.5), 
                         nn.Linear(100,2))

Using cache found in /dartfs-hpc/rc/home/9/f003xr9/.cache/torch/hub/pytorch_vision_main


In [20]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [21]:
#load the best model 
model_path = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/Saved_Models/resnet50.pt"
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [22]:
#modify the model so that the output is the embedding layer 
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

model.fc = Identity() # remove the fc layer 
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [23]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

## Create Embeddings

In [24]:
#get the patches
path = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/scc_tumor_data/prelim_patch_info/109_A1c_ASAP_tumor_map.npy"
arr = np.load(path)

In [25]:
embed_dic = {} #map the patch_id -> embedding
patches = []

In [26]:
for patch_id in tqdm(range(0, len(arr))):
    patch = preprocess(arr[patch_id])
    patches.append((patch_id, patch))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41295/41295 [00:13<00:00, 3063.88it/s]


In [27]:
patches_loader = DataLoader(dataset = patches, batch_size = 1)

In [28]:
with torch.no_grad():
    for idx, patch in tqdm(patches_loader):
        patch = patch.to(device=device)
        embed = model(patch)
        embed_dic[idx.item()] = embed.detach().cpu().tolist()[0]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41295/41295 [07:30<00:00, 91.56it/s]


In [29]:
len(embed_dic)
embeds = []

In [30]:
for patch_idx in embed_dic:
    embeds.append(embed_dic[patch_idx])

In [31]:
embeds = torch.tensor(embeds)

In [32]:
embeds.shape

torch.Size([41295, 2048])

In [33]:
adj_list = adj_list.T

In [34]:
y = torch.tensor(list(df["scc"])) # this is the scc for each patch 

In [35]:
y.shape

torch.Size([41295])

In [36]:
from torch_geometric.data import Data

In [37]:
data = Data(x=embeds, edge_index=adj_list, y=y)

In [38]:
#see if you can save this object 
torch.save(data, "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/graph_data/109_A1c_ASAP.pt")

In [39]:
#see if you can load it 
recovered_data = torch.load("/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/graph_data/109_A1c_ASAP.pt")

In [40]:
recovered_data

Data(x=[41295, 2048], edge_index=[2, 326540], y=[41295])

In [41]:
#combine all of this into a function

#takes: df of WSI, np array of WSI, and model 
#returns: saves the graph data in a directory 
def create_graph(df, arr, model, preprocess, device, save_dir, sample_id):
    
    #create node -> (x,y) map 
    pos_map = {}
    for index, row in tqdm(df.iterrows()):
        pos_map[index] = (row["x"], row["y"])
        
    #use these nodes, which are made sequentially, to create a graph and eventually and edge list
    nodes = torch.tensor([])
    for i in tqdm(pos_map):
        x = torch.tensor([[float(pos_map[i][0]), float(pos_map[i][1])]])
        nodes = torch.cat((nodes, x), 0)
    
    # create graph where points sqrt(2)*256 away from each other are considered neighbors 
    r_g, dist = dgl.radius_graph(nodes, 256*(2**(1/2)), get_distances=True) 
    
    #get the adj_list
    adj_list = list(zip(list(r_g.edges()[0]), list(r_g.edges()[1])))  #this is the adj list 
    for i in tqdm(range(len(adj_list))): #converting everything from tensors to ints 
        adj_list[i] = [adj_list[i][0].item(), adj_list[i][1].item()]
        
    #make it a tensor
    adj_list = torch.tensor(adj_list).T
    
    #now, create embeddings for all of the patches within the WSI 
    embed_dic = {} #map the patch_id -> embedding
    patches = []
    
    for patch_id in tqdm(range(0, len(arr))): #get (idx, patch array)
        patch = preprocess(arr[patch_id])
        patches.append((patch_id, patch))

    patches_loader = DataLoader(dataset = patches, batch_size = 1)

    with torch.no_grad():
        for idx, patch in tqdm(patches_loader): # get the embeddings here 
            patch = patch.to(device=device)
            embed = model(patch)
            embed_dic[idx.item()] = embed.detach().cpu().tolist()[0]
    
    #now create an array for these embeddings 
    embeds = []
    for patch_idx in embed_dic:
        embeds.append(embed_dic[patch_idx])
    embeds = torch.tensor(embeds)
    
    #get the SCC array 
    y = torch.tensor(list(df["scc"])) # this is the scc for each patch 
    
    #make the graph data object 
    data = Data(x=embeds, edge_index=adj_list, y=y)
    
    #save this object 
    print(torch.save(data, save_dir+sample_id+".pt"))

In [42]:
#test function 
save_dir = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/graph_data/"
sample_id = "109_A1c_ASAP"

create_graph(df = df, arr= arr, model = model, preprocess=preprocess, device=device, save_dir = save_dir, sample_id = sample_id)

41295it [00:01, 30528.92it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41295/41295 [07:23<00:00, 93.18it/s]


None


# Create & Save Graphs From All of the WSI 

In [47]:
metadata = pd.DataFrame()
#here we will store the sample ids and their paths 
samples = []
paths = []

In [48]:
#now create an np array containing all of the included patches from the relevant tumor maps
parent_dir = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/scc_tumor_data/prelim_patch_info/"
save_dir = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/graph_data/"

df_dir = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/scc_tumor_data/prelim_patch_info_v2/"

In [ ]:
#iterate through all of the WSI samples
for f in tqdm(os.listdir(parent_dir)):
    id, ext = f.split(".")
    if ext == "npy":
        id = id[0:id.find("tumor")-1]
        print(id)

        #load the df for this sample 
        df = pd.read_pickle(df_dir + id + "_tumor_map.pkl")

        #load the np array for this sample 
        arr = np.load(parent_dir + id + "_tumor_map.npy")
        
        #create and save graph 
        create_graph(df = df, arr= arr, model = model, preprocess=preprocess, device=device, save_dir = save_dir, sample_id = id)
        
        #append sample_id and path 
        samples.append(id)
        paths.append(save_dir + id + ".pt")
        

  0%|                                                                                                                                                                               | 0/190 [00:00<?, ?it/s]

109_A1c_ASAP



0it [00:00, ?it/s]
2994it [00:00, 29933.25it/s]
6014it [00:00, 30089.74it/s]
9036it [00:00, 30146.31it/s]
12051it [00:00, 30036.03it/s]
15065it [00:00, 30070.63it/s]
18080it [00:00, 30095.07it/s]
21097it [00:00, 30117.86it/s]
24109it [00:00, 30023.72it/s]
27116it [00:00, 30037.11it/s]
30218it [00:01, 30339.37it/s]
33279it [00:01, 30419.39it/s]
36322it [00:01, 30322.90it/s]
41295it [00:01, 30147.07it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41295/41295 [00:01<00:00, 38755.48it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 326540/326540 [00:00<00:00, 664139.96it/s]

 19%|██████████████████████████████▎                                                                                                                                 | 7829/41295 [0

 51%|█████████████████████████████████████████████████████████████████████████████████▏                                                                             | 21095/41295 [00:06<00:05, 3384.75it/s]


 84%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                          | 34491/41295 [00:10<00:01, 3636.02it/s]


...

  6%|█████████▍                                                                                                                                                        | 1289/22247 [00:13<03:50, 90.83it/s]


  8%|████████████▏                                                                                                                                                     | 1679/22247 [00:18<03:34, 96.02it/s]


  9%|███████████████                                                                                                                                                   | 2069/22247 [00:22<03:32, 94.85it/s]


 11%|█████████████████▉                                                                                                                                                | 2459/22247 [00:26<03:27, 95.49it/s]


 12%|████████████████████                                                                                                                                              | 2749/22247 [00:29<03:23, 95.61it/s]


In [ ]:
metadata["sample_id"] = samples
metadata["path"] = paths

In [ ]:
metadata.to_csv("/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/SCC-Tumor-Detection/Gokul_files/graph_data/metadata.csv")

